# SqlAlchemy

SQLAlchemy es una biblioteca de Python que proporciona una interfaz flexible y eficiente para interactuar con bases de datos SQL. Es una de las herramientas más utilizadas en la industria para manejar bases de datos de manera programática. Principalmente existen dos usos:
* SQLAlchemy Core: Usa SQL nativo y consultas directas con engine y connection.
* SQLAlchemy ORM (Object-Relational Mapping): Usa clases y objetos Python para representar tablas y manejar datos de forma estructurada.

In [ ]:
from sqlalchemy import create_engine, Column, String
from sqlalchemy.orm import declarative_base, sessionmaker
import sqlalchemy
import numpy as np
import pandas as pd
import polars as pl

## ORM

In [ ]:
# Conectar a DuckDB en memoria
engine = create_engine("duckdb:///../data/database.db")

# Base para definir las tablas
Base = declarative_base()


# Definir la tabla como una clase en Python
class RegionXCiudad(Base):
    __tablename__ = "REGION_X_CIUDAD"
    __table_args__ = {"schema": "DATMLOPS"}
    REGION = Column(String, primary_key=True)
    CIUDAD = Column(String)


# Crear la tabla en la base de datos
Base.metadata.create_all(engine)

# Crear sesión para interactuar con la BD
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
results = session.query(RegionXCiudad)
for result in results.yield_per(2):
    print(vars(result))

In [ ]:
df = pd.read_sql(session.query(RegionXCiudad).statement, session.bind, chunksize=2)
ls_lf = []
for df_iter in df:
    ls_lf.append(pl.from_pandas(df_iter).lazy())

lf = pl.concat(ls_lf, how="vertical")
lf.collect()

In [ ]:
df = lf.collect()
df = df.to_pandas()

In [ ]:
dtypes_df = dict(df.dtypes)
for k, v in dtypes_df.items():
    if v == np.int64:
        dtypes_df[k] = sqlalchemy.types.BigInteger()
    elif v == np.float64:
        dtypes_df[k] = sqlalchemy.types.Float()
    elif isinstance(v, object):
        dtypes_df[k] = sqlalchemy.types.String(df[k].str.len().max() + 10)
    else:
        raise

df.to_sql(
    name=r"REGION_X_CIUDAD_AUX".upper(),
    schema="DATMLOPS",
    con=session.bind,
    index=False,
    dtype=dtypes_df,
    chunksize=1000,
    # if_exists="append",
    if_exists="replace",
)

In [ ]:
df = pd.read_sql(
    """select * from DATMLOPS.REGION_X_CIUDAD_AUX""", session.bind, chunksize=2
)
ls_lf = []
for df_iter in df:
    ls_lf.append(pl.from_pandas(df_iter).lazy())

lf = pl.concat(ls_lf, how="vertical")
lf.collect()

In [ ]:
session.close()
engine.dispose()